In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

start = '2010-01-01'
end = '2024-09-14'

import yfinance as yf

df = yf.download('TTML.NS', start=start, end=end)
print(df.head())


In [ ]:
df = df.reset_index()
df.head()

In [ ]:
df = df.drop(['Adj Close'], axis=1)
df.head()

In [ ]:
ma100 = df.Close.rolling(100).mean()
ma100

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df.Close)
plt.plot(ma100, 'r')
plt.show()
#

In [ ]:
ma200 = df.Close.rolling(200).mean()
ma200

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df.Close)
plt.plot(ma100, 'r')
plt.plot(ma200, 'g')
plt.show()

In [ ]:
df.shape

In [ ]:
# splitting data into training and testing
data_training = pd.DataFrame(df['Close'][0:int(len(df) * 0.70)])
data_testing = pd.DataFrame(df['Close'][int(len(df) * 0.70):int(len(df))])

print(data_training.shape)
print(data_testing.shape)

In [ ]:
data_training.head()


In [ ]:
data_testing.head()

In [12]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))

In [ ]:
data_training_array = scaler.fit_transform(data_training)
data_training_array

In [14]:
x_train = []
y_train = []

for i in range(100, data_training_array.shape[0]):
    x_train.append(data_training_array[i - 100:i])
    y_train.append(data_training_array[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)

In [15]:
# ML Model

In [16]:
from keras.layers import Dense, Dropout, LSTM
from keras.models import Sequential

In [ ]:
model = Sequential()
model.add(LSTM(units=50, activation='relu', return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))


model.add(LSTM(units=60, activation='relu', return_sequences=True))
model.add(Dropout(0.3))

model.add(LSTM(units=80, activation='relu', return_sequences=True))
model.add(Dropout(0.4))

model.add(LSTM(units=120, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(units=1))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=50)

In [ ]:
model.save('keras_model.h5')

In [ ]:
data_testing.head()

In [ ]:
past_100_days = data_training.tail(100)

In [ ]:
# final_df = past_100_days.append(data_testing, ignore_index=True)

final_df = pd.concat([past_100_days, data_testing], ignore_index=True)

In [ ]:
final_df.head()

In [ ]:
final_df.shape

In [ ]:
input_data = scaler.fit_transform(final_df)
input_data

In [ ]:
input_data.shape

In [ ]:
x_test = []
y_test = []

In [ ]:
for i in range(100, input_data.shape[0]):
    x_test.append(input_data[i - 100:i])
    y_test.append(input_data[i, 0])

In [ ]:
x_test, y_test = np.array(x_test), np.array(y_test)
print(x_test.shape)
print(y_test.shape)

In [ ]:
# Making Prediction

In [ ]:
y_predicted = model.predict(x_test)

In [ ]:
y_predicted.shape

In [ ]:
y_test

In [ ]:
y_predicted

In [ ]:
scaler.scale_

In [ ]:
scaler_factor = 1 / 0.02123255
y_predicted = y_predicted * scaler_factor
y_test = y_test * scaler_factor

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(y_test, 'b', label='Original Price')
plt.plot(y_predicted, 'r', label='Predicted Price')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()